<a href="https://colab.research.google.com/github/dohyun1411/Quora-Insincere-Questions-Classification/blob/preprocessing1/clean_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clean text

reference: https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings#

In the reference, he found embeddings(Goolgle News) for 60.43% of vocab and 98.96% of all text.

additional reference: https://www.kaggle.com/wowfattie/3rd-place

In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

import tensorflow as tf

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Please change 'input_path' to the path where the input data is.

In [2]:
import os

# input_path = '/kaggle/input/quora-insincere-questions-classification'
input_path = './'
train_path = os.path.join(input_path, 'train.csv')
test_path = os.path.join(input_path, 'test.csv')
embeddings_path = os.path.join(input_path, 'embeddings.zip')

In [3]:
if os.path.exists(train_path): train = pd.read_csv(train_path)
else: train = pd.read_csv(train_path + '.zip')

if os.path.exists(test_path): test = pd.read_csv(train_path)
else: test = pd.read_csv(train_path + '.zip')

print('train shape:', train.shape)
print(train.head())
print()
print('test shape:', test.shape)
print(test.head())

train shape: (1306122, 3)
                    qid  ... target
0  00002165364db923c7e6  ...      0
1  000032939017120e6e44  ...      0
2  0000412ca6e4628ce2cf  ...      0
3  000042bf85aa498cd78e  ...      0
4  0000455dfa3e01eae3af  ...      0

[5 rows x 3 columns]

test shape: (1306122, 3)
                    qid  ... target
0  00002165364db923c7e6  ...      0
1  000032939017120e6e44  ...      0
2  0000412ca6e4628ce2cf  ...      0
3  000042bf85aa498cd78e  ...      0
4  0000455dfa3e01eae3af  ...      0

[5 rows x 3 columns]


In [4]:
train_text = train['question_text']
test_text = test['question_text']

In [5]:
from collections import defaultdict

def build_vocab(sentences, verbose=1):
    vocab = defaultdict(int)
    for sentence in tqdm(sentences, disable=(not verbose)):
        for word in sentence:
            vocab[word] += 1
    return vocab

In [6]:
%%time
import zipfile
from gensim.models import KeyedVectors

google = 'GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'

with zipfile.ZipFile(embeddings_path) as embedding_zip:
    print("Found embeddings as a zip file")
    google_embeddings = KeyedVectors.load_word2vec_format(embedding_zip.open(google), binary=True)

Found embeddings as a zip file
CPU times: user 2min 15s, sys: 4.74 s, total: 2min 20s
Wall time: 2min 20s


In [7]:
import re

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [8]:
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


mispell_dict = {'colour':'color',
                'centre':'center',
                'didnt':'did not',
                'doesnt':'does not',
                'isnt':'is not',
                'shouldnt':'should not',
                'favourite':'favorite',
                'travelling':'traveling',
                'counselling':'counseling',
                'theatre':'theater',
                'cancelled':'canceled',
                'labour':'labor',
                'organisation':'organization',
                'wwii':'world war 2',
                'citicise':'criticize',
                'instagram': 'social medium',
                'whatsapp': 'social medium',
                'snapchat': 'social medium'
                }
mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

In [9]:
import operator

def check_coverage(voca, embeddings, verbose=1):
    
    inv = {}
    oov = {}
    count_inv = 0
    count_oov = 0
    
    for word in tqdm(voca, disable=(not verbose)):
        
        if word in embeddings:
            inv[word] = embeddings[word]
            count_inv += voca[word]
        
        else:
            oov[word] = voca[word]
            count_oov += voca[word]

    print('Found embeddings for {:.2%} of voca.'.format(len(inv) / len(voca)))
    print('Found embeddings for {:.2%} of all text.'.format(count_inv / (count_inv + count_oov)))

    return sorted(oov.items(), key=operator.itemgetter(1))[::-1]

Using spaCy

In [10]:
import spacy

# !pip install -U spacy[cuda100]
gpu = spacy.prefer_gpu()
print('GPU:', gpu)

# !pip install -U pip setuptools wheel
# !pip install -U spacy
# !python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner', 'tagger'])

GPU: True


In [11]:
train_text = train_text.progress_apply(replace_typical_misspell)

In [13]:
docs = nlp.pipe(train_text)

print("hello")

hello
